In [1]:
# !pip install langchain
# !pip install openai

In [2]:
# !pip install langchain_community

In [3]:
# pip install openai==0.28.0

In [4]:
import os

from langchain.llms import OpenAI

from langchain.chat_models import ChatOpenAI

In [5]:
from google.colab import userdata
import openai
os.environ["OPENAI_API_KEY"] = "api key 입력"

In [25]:
davinch3 = OpenAI(
    model_name="text-davinci-003",
    max_tokens = 1000
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [6]:
chatgpt = OpenAI(
    model_name = 'gpt-3.5-turbo',
    max_tokens = 1000
)

/usr/local/lib/python3.10/dist-packages/langchain_community/llms/openai.py:253: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain_community.chat_models import ChatOpenAI`
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain_community/llms/openai.py:1076: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain_community.chat_models import ChatOpenAI`
  warnings.warn(


## 프롬프트 탬플릿은 크게 2가지가 존재

1. Prompt Template
2. Chat Prompt Template

- Prompt Template : 일반적인 프롬프트 템플랫을 생성할때 활용
- Chat Prompt Template : 채팅 LLM에 프롬프트를 전달하는 데에 활용할 수 있는 특화 프롬프트 템플릿

### Prompt Template

In [7]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate

#프롬프트 템플릿을 통해 매개변수 삽입이 가능한 문자열로 반환
string_prompt = PromptTemplate.from_template("tell me a joke about {subject}")

#매개변수 삽입한 결과를 string_prompt_value에 할당
string_prompt_value = string_prompt.format_prompt(subject = 'soccer')

#채팅 LLM이 아닌 LLM과 대화할 때 필요한 프롬프트 = string_prompt
string_prompt_value

StringPromptValue(text='tell me a joke about soccer')

In [8]:
#to_string() 함술르 통해 prompt template로 생성한 문자 반환
string_prompt_value.to_string()

'tell me a joke about soccer'

### ChatPromptTemplate

In [9]:
chat_prompt = ChatPromptTemplate.from_template("tell me a joke about {subject}")
chat_prompt_value = chat_prompt.format_prompt(subject = 'soccer')
chat_prompt_value

#사용자가 보내는 메시지로 나옴

ChatPromptValue(messages=[HumanMessage(content='tell me a joke about soccer')])

In [10]:
chat_prompt_value.to_string()

'Human: tell me a joke about soccer'

# 프롬프트 템플릿 활용해보기

### 반복적인 프롬프트를 삽입해야 하는 경우, Prompt Template를 통해 간편하게 LLM을 활용할 수 있다.

- GPT-3.5와 프롬프트 템플릿을 활용하여 대화해보기

In [11]:
from langchain.prompts.prompt import PromptTemplate

template = """
너는 요리사야. 내가 가진 재료들을 가지고 만들 수 있는 요리를 추천하고, 그 요리의 레시피를 제시해줘.
내가 가진 재료는 아래와 같아.

<재료>
{재료}

"""
prompt_template = PromptTemplate(
    input_variables = ['재료'],
    template = template
)

In [12]:
print(prompt_template.format(재료 = '양파, 계란, 사과, 빵'))


너는 요리사야. 내가 가진 재료들을 가지고 만들 수 있는 요리를 추천하고, 그 요리의 레시피를 제시해줘.
내가 가진 재료는 아래와 같아. 

<재료>
양파, 계란, 사과, 빵




In [13]:
print(chatgpt(
    prompt_template.format(재료 = '양파, 계란, 사과, 빵')
))

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


제가 추천하는 요리는 "양파 계란 말이" 입니다. 

[양파 계란 말이 레시피]
재료:
- 양파 1개
- 계란 2개
- 소금 1/2 작은술
- 후추 약간

1. 양파를 얇게 채썰어 준비합니다.
2. 계란을 그릇에 깨고 소금과 후추를 넣고 푼 후에 양파를 넣어 섞어줍니다. 
3. 팬에 식용유를 두르고 양파 계란을 부어 잘 익혀줍니다. 
4. 불을 끄고 완성된 요리 위에 깨끗한 빵을 곁들여 발라 먹습니다. 

맛있게 즐기세요! 😊


- ChatGPT와 프롬프트 템플릿을 활용하여 대화해보기

In [15]:
from langchain.prompts import (ChatPromptTemplate, PromptTemplate, SystemMessagePromptTemplate,
                              AIMessagePromptTemplate, HumanMessagePromptTemplate)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [16]:
#chatgpt 모델 로드
chatgpt = ChatOpenAI(temperature=0)

#chatgpt에게 역할 부여
system_message_prompt = SystemMessagePromptTemplate.from_template(template)

#사용자가 입력할 매개변수 template을 선언
human_template = "{재료}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

#ChatPromptTemplate에 system message와 human message 템플릿을 삽입
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

answer = chatgpt(chat_prompt.format_prompt(재료 = '양파, 계란, 사과, 빵').to_messages())
print(answer.content)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


제가 추천하는 요리는 "양파 계란 토스트"입니다. 맛있고 간단하게 만들 수 있는 요리에요. 

**양파 계란 토스트 레시피:**

**재료:**
- 양파
- 계란
- 빵
- 소금
- 후추
- 식용유

**만드는 법:**
1. 양파를 얇게 슬라이스해줍니다.
2. 팬에 식용유를 두르고 양파를 볶아 투명해질 때까지 볶아줍니다.
3. 볶은 양파에 소금과 후추를 넣고 볶아줍니다.
4. 계란을 풀어 소금을 약간 넣고 푸짐하게 풀어줍니다.
5. 다른 팬에 식용유를 두르고 계란을 부칩니다.
6. 계란이 익으면 양파와 섞어줍니다.
7. 빵을 굽거나 토스터에 구워서 바삭하게 구워줍니다.
8. 바삭한 빵 위에 양파와 계란을 올려줍니다.

맛있는 양파 계란 토스트가 완성되었어요. 맛있게 즐기세요!


##Few-Shot 예제를 통한 프롬프트 템플릿

- Few-Shot이란, 딥러닝 모델이 결과물을 출력할 때 예시 결과물을 제시함으로써 원하는 결과물로 유도하는 방법론
- LLM역시, Few-Shot예제를 제공하면 예제와 유사한 형태의 결과물을 출력

- 내가 원하는 결과물의 형태가 특수하거나, 구조화된 답변을 원할 경우, 결과물의 예시를 수 개 제시함으로써 결과물의 품질을 향상시킬 수 있다

In [17]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate

examples = [
    {
        "question" : '에스파로 삼행시 만들어줘',
        "answer" :
"""
에 : 에라이
스 : 스파게티에
파 : 파를 너무 많이 넣었어.
"""
    },
    {
       "question" : '이예진으로 삼행시 만들어줘',
        "answer" :
"""
이 : 이것 좀 봐바
예 : 예상도 못했어
진 : 진짜
"""
    }
]

In [29]:
example_prompt = PromptTemplate(input_variables = ["question", "answer"], template = "Question: : {question}\n{answer}")

print(example_prompt.format(**examples[0]))

Question: : 에스파로 삼행시 만들어줘

에 : 에라이
스 : 스파게티에 
파 : 파를 너무 많이 넣었어.



In [30]:
prompt = FewShotPromptTemplate(
    examples = examples, #출력
    example_prompt = example_prompt,
    suffix = "Question : {input}",
    input_variables = ["input"]
)

print(prompt.format(input = '호날두으로 삼행시 만들어줘'))

Question: : 에스파로 삼행시 만들어줘

에 : 에라이
스 : 스파게티에 
파 : 파를 너무 많이 넣었어.


Question: : 이예진으로 삼행시 만들어줘

이 : 이것 좀 봐바 
예 : 예상도 못했어
진 : 진짜


Question : 호날두으로 삼행시 만들어줘


In [31]:
print(chatgpt.predict("호날두으로 삼행시 만들어줘"))

호날두가 뛰어다니며
공을 차고 놀아요
축구왕이 되리라


In [33]:
print(davinch3(
    prompt.format(input = "호날두로 삼행시 만들어줘")
))

InvalidRequestError: The model `text-davinci-003` has been deprecated, learn more here: https://platform.openai.com/docs/deprecations

text-davinci-003 모델 지원 x